# Seminar 11-12



*   **Supervised Learning** - we have target (correct answer), model is trained to predict it. Regression, classification are supervised methods. Examples: predict prices of houses, predict if the passenger will survive Titanic accident, etc
*   **Unsupervised Learning** - no right answer, just input data. Goal is to find some dependencies, groups, etc. Example: information about students of HSE - what can be said about them? What groups do we have?



# Clustering

CLustering is a unsupervised method. Main idea is to separate data into groups (cluster), each cluster should contain similar samples and different clusters should contain different samples.

Let's start with the example. How many groups of data do we seem to have? Which point is in which group?

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

#Generate some data
np.random.seed(123)
X1 = np.random.randn(100,2)
X2 = np.random.randn(100,2) - np.array([10,1])
X3 = np.random.randn(100,2) - np.array([1,10])
X = np.vstack((X1,X2,X3))

plt.scatter(X[:, 0], X[:,1])

## KMeans

### Idea

KMeans is a clustering algorithm. It has a hyperparameter $k$ (number of clusters)

The algorithm:



1.   Initialize $k$ cluster centers randomly.
2.   Assign points to corresponding clusters (with minimum distance to their center).
3. Recalculate cluster centers using mean for all points belonging to the cluster.
4. Steps 2-3 are repeated until cluster centers stop changing (significantly).

In [ ]:
k = 3 #Let's find 3 groups/clusters

Let's implement it:

*Step 1* initialize cluster centers.

In [ ]:
np.random.seed(0)

centers = np.random.uniform(low=X.min(),high=X.max(),size=(k, X.shape[1]))

plt.scatter(X[:, 0], X[:,1], label='data')
plt.scatter(centers[:, 0], centers[:,1], label='random centers', color='black')
plt.legend()

*Step 2* Assign points to the nearest cluster

In [ ]:
from scipy.spatial.distance import cdist

#for every observation compute the distance to the center
dists = cdist(X, centers)

dists.shape

In [ ]:
labels = dists.argmin(axis=1)
labels.shape

In [ ]:
for cluster in sorted(np.unique(labels)):
  plt.scatter(X[labels == cluster][:, 0], X[labels == cluster][:,1], label='cluster ' + str(cluster))
plt.scatter(centers[:, 0], centers[:,1], label='centers', color='black', marker='x')
plt.legend()

*Step 3* Recalculate centers

In [ ]:
for cluster in sorted(np.unique(labels)):
  centers[cluster] = np.mean(X[labels == cluster], axis=0)

for cluster in sorted(np.unique(labels)):
  plt.scatter(X[labels == cluster][:, 0], X[labels == cluster][:,1], label='cluster ' + str(cluster))
plt.scatter(centers[:, 0], centers[:,1], label='recalculated centers', color='black', marker='x')
plt.legend()

*Step 4* - Repeat steps 2-3

In [ ]:
#for every observation compute the distance to the center
dists = cdist(X, centers)
#set the new labels
labels = dists.argmin(axis=1)

#plot new clusters
for cluster in sorted(np.unique(labels)):
  plt.scatter(X[labels == cluster][:, 0], X[labels == cluster][:,1], label='cluster ' + str(cluster))
plt.scatter(centers[:, 0], centers[:,1], label='centers', color='black', marker='x')
plt.legend()

In [ ]:
#recalculate centers
for cluster in sorted(np.unique(labels)):
  centers[cluster] = np.mean(X[labels == cluster], axis=0)

#plot old clusters and new centers
for cluster in sorted(np.unique(labels)):
  plt.scatter(X[labels == cluster][:, 0], X[labels == cluster][:,1], label='cluster ' + str(cluster))
plt.scatter(centers[:, 0], centers[:,1], label='recalculated centers', color='black', marker='x')
plt.legend()

The recalculated centers do not differ much from the ones on the previous steps. Seems like we are almost finished

In [ ]:
#for every observation compute the distance to the center
dists = cdist(X, centers)
#set the new labels
labels = dists.argmin(axis=1)

#recalculate centers
for cluster in sorted(np.unique(labels)):
  centers[cluster] = np.mean(X[labels == cluster], axis=0)

#plot old clusters and new centers
for cluster in sorted(np.unique(labels)):
  plt.scatter(X[labels == cluster][:, 0], X[labels == cluster][:,1], label='cluster ' + str(cluster))
plt.scatter(centers[:, 0], centers[:,1], label='recalculated centers', color='black', marker='x')
plt.legend()

The new clustering does not differ at all, all the points have the same labels (points did not change colors) and centers are in the same place. We should stop the algorithm.

KMeans divided data into 3 clusters and the division is nice.

Now, let's check KMeans from sklearn.

In [ ]:
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters = 3)
#Fit using only X (there are no target/y)
k_means = k_means.fit(X)
clusters = k_means.predict(X)


plt.scatter(X[:,0], X[:,1], c = clusters)
plt.title('KMeans clustering')
plt.show()

What if we did not set the number of clusters ($k$) correctly

In [ ]:
plt.figure(figsize= (15,8))
for n_c in range(2,8):
    k_means = KMeans(n_clusters = n_c)
    k_means = k_means.fit(X)
    clusters = k_means.predict(X)
    plt.subplot(2,3,n_c - 1)
    plt.scatter(X[:,0], X[:,1], c = clusters)
    plt.title('n_clusters = {}'.format(n_c))

plt.show()

What can we see:


*   KMeans tries to assign points to every cluster (it cannot reduce/increase number of clusters)
*   Clusters are convex (looks like circles, ovals, spheres in multidimensional space)
*   Clusters have similar sizes (number of points)



The advantage of the algorith is the speed (one iteration - just recalculating distances)

The disdvantages:


*   Random initialization of cluster centers (may get different clusters)
*   Assumptions about cluster shapes, sizes



[Visualization of KMeans](https://www.naftaliharris.com/blog/visualizing-k-means-clustering/)

### Clustering metrics

How to search for hyperparameters? We do not have target to assess the quality.

Possible approach is to use the internal clustering validation measures (just try to measure, how good are the groups)

We need to measure:

* cohesion - the closer samples in one cluster, the better
* separation - the further different clusters are from each other, the better



For example,  Silhouette Coefficient:

$$Sil(X, C) = \frac{1}{|C|}\sum_{c_k \in C}\frac{1}{|c_k|}\sum_{x_i \in c_k}\frac{separation(x_i, c_k) - cohension(x_i, c_k)}{max\{separation(x_i, c_k) , cohension(x_i, c_k)\}}$$

where

$separation(x_i, c_k) = min_{c_l \in C ∖ \{c_k\}} \{\frac{1}{|c_l|}\sum_{x_j \in c_l}||x_i - x_j||\}$ - mean distance to the object of the closest cluster;


$cohension(x_i, c_k) = \frac{1}{|c_k|-1}\sum_{x_j \in c_k}||x_i - x_j||$ - mean distance to the objects of the same cluster


$-1 \le Sil(X, C) \le 1$

The closer it to 1, the better. If it is near 0, the clusters are overlapping (same disnace from the point to the closest and its own cluster). Negative score means that samples have wrong labels (objects are closer to other cluster than to its own cluster)


In [ ]:
from sklearn.metrics import silhouette_score

best_k, best_score = None, -1
for k in range(2,15):
    k_means = KMeans(n_clusters = k)
    k_means = k_means.fit(X)
    clusters = k_means.predict(X)
    score = silhouette_score(X=X,
                             labels=clusters)
    print(k, score)
    if score > best_score:
      best_score = score
      best_k = k
print('Best score {}, k = {}'.format(best_score, best_k))

### Fails

Let's check the data, where KMeans fail.

The data looks like two moons. Most likely, we need to separate data int 2 clusters: upper and lower moon

In [ ]:
from sklearn.datasets import make_moons, make_circles
X_moon, y_moon = make_moons(n_samples=500, noise=0.1) #y_moon contain info about what moon the point belong to in reality

plt.scatter(X_moon[:,0], X_moon[:,1])
plt.show()

In [ ]:
k_means = KMeans(n_clusters = 2)
k_means = k_means.fit(X_moon)
clusters = k_means.predict(X_moon)

plt.scatter(X_moon[:,0], X_moon[:,1], c=clusters)
plt.show()

KMeans separated convex clusters  Let's look where the centers are:

In [ ]:
plt.scatter(X_moon[:,0], X_moon[:,1], c=clusters)

for cl in np.unique(clusters):
  center_ = X_moon[clusters == cl].mean(axis=0)
  plt.scatter([center_[0]], [center_[1]], label='Center cluster ' + str(cl), marker='x')

plt.legend()
plt.show()

Why did KMeans fail? Because clusters are non-convex and the part of one cluster is closer to the centriod of another. KMeans does not allow it.

In [ ]:
#Reality
plt.scatter(X_moon[:,0], X_moon[:,1], c=y_moon)

for cl in np.unique(y_moon):
  center_ = X_moon[y_moon == cl].mean(axis=0)
  plt.scatter([center_[0]], [center_[1]], label='Center of moon ' + str(cl), marker='x')

plt.legend()
plt.show()

Another example:

In [ ]:
from sklearn.datasets import make_circles

X_circle, y_circle = make_circles(n_samples=500, noise=0.05, factor=0.5)
plt.scatter(X_circle[:,0], X_circle[:,1])
plt.show()

Again, seems to be 2 clusters: inner and outer ring

In [ ]:
k_means = KMeans(n_clusters = 2)
k_means = k_means.fit(X_circle)
clusters = k_means.predict(X_circle)
plt.scatter(X_circle[:,0], X_circle[:,1], c=clusters)

for cl in np.unique(clusters):
  center_ = X_circle[clusters == cl].mean(axis=0)
  plt.scatter([center_[0]], [center_[1]], label='Center cluster ' + str(cl), marker='x')

plt.legend(loc='lower left')
plt.show()

In [ ]:
#Reality
plt.scatter(X_circle[:,0], X_circle[:,1], c=clusters)

for cl in np.unique(y_circle):
  center_ = X_circle[y_circle == cl].mean(axis=0)
  plt.scatter([center_[0]], [center_[1]], label='Center ring ' + str(cl), marker='x')

plt.legend(loc='lower left')
plt.show()

And another one:

In [ ]:
from sklearn.datasets import make_blobs

X_blob, y_blob = make_blobs(centers=[[-10, 0], [10, 0]], n_samples=100)
X_blob[0] = [200, 0]

plt.scatter(X_blob[:,0], X_blob[:,1])

Here we have 2 blobs (depicted as thin ovals) and one outlier.

In [ ]:
k_means = KMeans(n_clusters = 2)
k_means = k_means.fit(X_blob)
clusters = k_means.predict(X_blob)
plt.scatter(X_blob[:,0], X_blob[:,1], c=clusters)

for cl in np.unique(clusters):
  center_ = X_blob[clusters == cl].mean(axis=0)
  plt.scatter([center_[0]], [center_[1]], label='Center cluster ' + str(cl), marker='x')

plt.legend(loc='lower left')
plt.show()

Here there is an outlier - point really far from other data. KMeans considers it to be a separate cluster (because it makes center of the cluster shift far right and there is no reason to add it to another cluster).

## DBSCAN

DBSCAN - Density-based spatial clustering of applications with noise

Idea: dense areas (areas where are a lot of points) are the clusters, we need to move along dense areas to find the edge of it (less dense area next to dense area). If there are points very far from the dense areas, this points are considered *noise* (points which do not belong to any cluster)

Hyperparameters:



*   `eps` - the redius of a sphere
*   `min_samples` - the minimum number of point in the area (sphere of radius `eps`) to consider the area dense



Idea of an algorithm:



1.  Look into some point, draw a sphere of radius `eps` and compute the number of points in it (including the point itself)
2.   If there are less than `min_samples` points, we need to search for another point
3. If there are more or equal points this point belong to a cluster, all its neighbours are in this cluster, too
4. Investigate neighbours of the point: if they also have enogh naighbours in the sphere, we should consider their neighbours, etc; if not enough - this points are the edge of the cluster (we do not consider their neighbours)
5. After we run out of neighbours, move to step 1
6. If we considered all the left points and there are no more clusters - mark points as noise



Consider the example: `min_samples=4`

<p><a href="https://commons.wikimedia.org/wiki/File:DBSCAN-Illustration.svg#/media/Файл:DBSCAN-Illustration.svg"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/af/DBSCAN-Illustration.svg/1200px-DBSCAN-Illustration.svg.png" alt="DBSCAN-Illustration.svg" width="450" height="450"> </a><br> <a href="//commons.wikimedia.org/wiki/User:Chire" title="User:Chire">Chire</a> &mdash; <span class="int-own-work" lang="ru"></span><a href="https://creativecommons.org/licenses/by-sa/3.0" title="Creative Commons Attribution-Share Alike 3.0">CC BY-SA 3.0</a>

[Visualization of DBSCAN](https://www.naftaliharris.com/blog/visualizing-dbscan-clustering/)


Let's apply DBSCAN from sklearn to our data from the beginning of the seminar.

In [ ]:
from sklearn.cluster import DBSCAN
import matplotlib as mpl

#function to get nice colors
def get_colors(clusters):
  n_clusters = len(np.unique(clusters[clusters != -1]))

  palette = mpl.colormaps['viridis'].resampled(n_clusters)(np.linspace(0, 1, n_clusters))
  colors = np.zeros((clusters.shape[0], 4))
  colors[clusters != -1] = palette[clusters[clusters != -1]]

  #Noise will be transparent grey
  colors[clusters == -1] = [0.5, 0.5, 0.5, 0.3]

  return colors

plt.figure(figsize= (15,20))
i = 1

for e in [0.2, 1, 3, 5, 10]:
    for samples in [2, 5, 40]:
        dbscan = DBSCAN(eps=e, min_samples=samples)
        clusters = dbscan.fit_predict(X)

        n_clusters = len(np.unique(clusters[clusters != -1]))
        colors = get_colors(clusters)

        plt.subplot(5, 3, i)

        plt.scatter(X[:,0], X[:,1], c=colors)
        plt.title('eps = {}, min_samples = {} -> {} clusters'.format(e, samples, n_clusters))
        i += 1


plt.show()

And the datasets, where KMeans failed:

In [ ]:
dbscan = DBSCAN(eps=0.2, min_samples=10)
clusters = dbscan.fit_predict(X_moon)

colors = get_colors(clusters)
plt.scatter(X_moon[:,0], X_moon[:,1], c=colors)
plt.title('eps = {}, min_samples = {} -> {} clusters'.format(dbscan.eps, dbscan.min_samples, len(np.unique(clusters[clusters != -1]))))
plt.show()

In [ ]:
dbscan = DBSCAN(eps=0.2, min_samples=10)
clusters = dbscan.fit_predict(X_circle)
colors = get_colors(clusters)
plt.scatter(X_circle[:,0], X_circle[:,1], c=colors)
plt.title('eps = {}, min_samples = {} -> {} clusters'.format(dbscan.eps, dbscan.min_samples, len(np.unique(clusters[clusters != -1]))))
plt.show()

In [ ]:
dbscan = DBSCAN(eps=2, min_samples=10)
clusters = dbscan.fit_predict(X_blob)
colors = get_colors(clusters)
plt.scatter(X_blob[:,0], X_blob[:,1], c=colors)
plt.title('eps = {}, min_samples = {} -> {} clusters'.format(dbscan.eps, dbscan.min_samples, len(np.unique(clusters[clusters != -1]))))
plt.show()

DBSCAN can manage these tricky examples. It does not use centers of clusters, so the non-convex clusters do not bother it.

## Hierarchical clustering

Approaches:



*   Agglomerative: This is a "bottom-up" approach: Beginning: each observation is a cluster. Iteration: 2 clusters are merged. End: 1 cluster.
*  Divisive: This is a "top-down" approach: Beginning: 1 cluster . Iteration: 1 cluster is divided in 2. End: each observation is a cluster.

Let's use agglomerative approach.

Idea: greedy build clusters

Algorithm:

1.   Initially every point is a separate cluster
2.   During iteration the closest clusters are merged into one.
3.   Repeat step 2 until there is only 1 cluster (which contains all points)



How to compute distance between clusters (group of points)?

For example, Ward's Method:


$$
\Delta = \sum_{x_i \in A \cup B}{(x_i-\bar{x})^2} - \sum_{x_i \in A}(x_i - \bar{a})^2 - \sum_{x_i \in B}(x_i - \bar{b})^2
$$


It may be useful to use dendrogram to check, how the clustering works:

In [ ]:
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

#Code to draw dendrogram:
#https://scikit-learn.org/1.5/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)


random_state = 170


# setting distance_threshold=0 ensures we compute the full tree.
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)

model = model.fit(X)
plt.title("Hierarchical Clustering Dendrogram")
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode="level", p=5)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

What is going on the plot:



*   x axis is an id of a point (or number of points in little cluster, if it is in brackets)
*   y axis is a distance between clusters
*  when 2 clusters are united П is drawn, uniting them, the place of horizontal line is the distance between the united clusters



How to choose the clustering? (Here we gat a lot of candidates)

Basic approaces:



*   Number of clusters. If we want particular number of clusters, we may set `n_clusters` hyperparameter (draw a line on the dendrogram such that it crosses particular number of vertical lines)
*   Distance. We can set `distance_threshold` if the distance for next cluster merging is more than it, we need to stop (just draw horizontal line with corresponding y)
*  "Jump" in dendrogram. If we can see a drastic increase in distnce (the horizontal line soars), that means we started uniting far clusters and we need to stop



In [ ]:
#choosing only 3 clusters
model = AgglomerativeClustering(n_clusters=3)

clusters = model.fit_predict(X)

plt.scatter(X[:,0], X[:,1], c = clusters)
plt.title('Agglomerative clustering')
plt.show()

In [ ]:
#set the distance threshold as 80
#need to set n_clusters to None (bacause we can use only one approach)
model = AgglomerativeClustering(distance_threshold=80, n_clusters=None)

clusters = model.fit_predict(X)

plt.scatter(X[:,0], X[:,1], c = clusters)
plt.title('Agglomerative clustering')
plt.show()

## Real data

Let's apply clustering to some new data

For example, let's try to work with pictures of hand-written digits (one picture - one digit)

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()

X, y = digits.data, digits.target
Im = digits.images

for i in range(20):
    plt.figure(figsize=(2,2))
    plt.imshow(Im[i], cmap='gray')
    plt.show()

How can we work with pictures? Black and write picture is actually a 2D matrix - every pixel is assosiated with 1 number (brightness)

In [ ]:
Im[i]

Zeros is black, 16 is white.

We can flatten our matrix:

In [ ]:
Im[i].flatten()

So a 8x8 picture is actually a vector of size 64 (the features are 'how bright is the pixel i')

In [ ]:
X.shape

We actually can build a classification model and get good accuracy:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


lr = LogisticRegression(max_iter=1000, C=1).fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_test, y_pred)

But let's try to apply clustering:

In [ ]:
km = KMeans(n_clusters=10, max_iter=100, n_init=1)
km.fit(X)

In [ ]:
print("Silhouette Coefficient:", silhouette_score(X, km.labels_, sample_size=1000))

The clustering metric does not seem to be very big, but it is greater than 0.

But can we compare clustering to the target (if we have it)? Can we use classification metrics?

In [ ]:
#Let's try:
accuracy_score(y, km.labels_)

The accuracy seems to be bad. Why? Because classification wants particular labels (we cannot just swap 1 and 0 inside the prediction, we would be wrong), but for clustering the label (number of the cluster) does not matter at all.

If we want to compare clustering labels to classification we need to use external clustering validation measures.


For example, V measure - it assess if the cluters contain same class and same class contains same clusters

In [ ]:
from sklearn.metrics import v_measure_score

print('V-measure:', v_measure_score(y, km.labels_))

So, the clustering and classification almost agree

Let's plot the clustering!

But how... (we have 64 features, we cannot draw such plots)

# Dimensionality reduction, visualization

Let's try to depict some random pairs of features

In [ ]:
plt.figure(figsize=(12,10))
plt.scatter(X[:, 0], X[:, 1], c=y,
            edgecolor='none', alpha=0.7, s=40,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.colorbar()
plt.title('MNIST. 2 features');

In [ ]:
plt.figure(figsize=(12,10))
plt.scatter(X[:, 3], X[:, 45], c=y,
            edgecolor='none', alpha=0.7, s=40,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.colorbar()
plt.title('MNIST. 2 features');

That's strange. Why? - the values are integers (0-16), so, the scatterplots will be sparse and the brightness of 1 pixel may not be important (all numbers may have particular pixel black/white, it's combination what matters).


So, we need to use some method, that will reduce the dimensionality of data saving as much info as possible

## PCA

This method is using linear algebra. It searches for some linear combination of features such that it preserves as much information as possible

In [ ]:
from sklearn.decomposition import PCA

model_pca = PCA(n_components=2)
X_reduced = model_pca.fit_transform(X)

plt.figure(figsize=(12,10))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y,
            edgecolor='none', alpha=0.7, s=40,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.colorbar()
plt.title('MNIST. PCA projection');

PCA can be used not only for visualization, but just for dimensionality reduction (get new shrinked data, use later)

In [ ]:
model_pca = PCA(n_components=10)
X_reduced = model_pca.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.3, random_state=0)


lr = LogisticRegression(max_iter=1000, C=1).fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy_score(y_test, y_pred)

## t-SNE

t-SNE is a method for visualizaion. It trains new vector representation in such way that similar objects have close vectors and dis-similar have different vectors with big probability. It is a non-linear method, used uaually only for visualization.

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_jobs=4, perplexity=30, random_state=42)
X_tsne = tsne.fit_transform(X)

plt.figure(figsize=(12,10))
plt.scatter(X_tsne[:, 0], X_tsne[:, 1], c=y,
            edgecolor='none', alpha=0.7, s=40,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))

plt.colorbar()
plt.title('MNIST. t-SNE projection');